In [124]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import json
from IPython.display import display


headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
}
TPLAYERS = 100

In [125]:
with open("players_1.html") as file:
    soup = BeautifulSoup(file, "html.parser")
table_players = soup.find(class_='tablesorter')
players_a = table_players.find_all('a')
players_with_space = []
players_without_space = []
ctr = 0
for player_a in players_a:
    if ctr % 2 == 0:
        players_with_space.append(player_a.get_text())
        players_without_space.append(player_a.get_text().replace(" ", ""))
    ctr += 1

with open("players_2.html") as file:
    soup = BeautifulSoup(file, "html.parser")
table_players = soup.find(class_='tablesorter')
players_a = table_players.find_all('a')
for player_a in players_a:
    if ctr % 2 == 0:
        players_with_space.append(player_a.get_text())
        players_without_space.append(player_a.get_text().replace(" ", ""))
    ctr += 1

In [126]:
players_with_space[:5], players_without_space[:5], len(players_without_space)

(['Jannik Sinner',
  'Alexander Zverev',
  'Carlos Alcaraz',
  'Novak Djokovic',
  'Daniil Medvedev'],
 ['JannikSinner',
  'AlexanderZverev',
  'CarlosAlcaraz',
  'NovakDjokovic',
  'DaniilMedvedev'],
 100)

In [127]:
players_without_space[90], players_with_space[90]

('TaroDaniel', 'Taro Daniel')

In [128]:
def get_webscrape_player_data(i, player):
    url = f"https://www.tennisabstract.com/cgi-bin/player-classic.cgi?p={player}&f=ACareerqq"
    r = requests.get(url, headers=headers)

    soup = BeautifulSoup(r.content, 'html.parser')

    with open(f"players_data/{i}_{player}.html", "w") as file:
        file.write(str(soup))

    # print(script_tag)

In [129]:
print(get_webscrape_player_data(1, players_without_space[1]))

None


In [130]:
def get_player_match_data(i):
    with open(f"players_data/{i}_{players_without_space[i]}.html") as file:
        soup = BeautifulSoup(file, 'html.parser')
    scripts = soup.find_all('script')

    matchmx = None
    for script in scripts:
        if 'var matchmx' in script.text:
            matchmx_pattern = re.search(r'var matchmx\s*=\s*(\[[\s\S]*?\]);', script.text)
            if matchmx_pattern:
                matchmx = matchmx_pattern.group(1)
                break

    matchmx = json.loads(matchmx)

    return matchmx
print(get_player_match_data(1))

[['20240920', 'Laver Cup', 'Hard', 'A', 'W', '2', '', '', 'RR', '6-7(5) 7-5 [10-5]', '3', 'Frances Tiafoe', '16', '', '', 'R', '19980120', '188', 'USA', '0', '154', '12', '0', '85', '63', '50', '10', '12', '1', '2', '7', '2', '96', '53', '39', '21', '12', '7', '9', '2', '', '', '', '2024-9210-398', '', '', '', '126207'], ['20240920', 'Laver Cup', 'Hard', 'A', 'L', '2', '', '', 'RR', '6-4 7-5', '3', 'Taylor Fritz', '7', '', '', 'R', '19971028', '196', 'USA', '0', '117', '3', '2', '69', '52', '31', '9', '11', '4', '7', '8', '3', '71', '38', '33', '16', '11', '4', '5', '2', '', '', '', '2024-9210-396', '', '', '', '126203'], ['20240826', 'US Open', 'Hard', 'G', 'L', '4', '4', '', 'QF', '7-6(2) 3-6 6-4 7-6(3)', '5', 'Taylor Fritz', '12', '12', '', 'R', '19971028', '196', 'USA', '0', '206', '14', '4', '145', '103', '77', '20', '22', '11', '13', '12', '5', '128', '81', '66', '25', '21', '2', '4', '2', '', '', '', '2024-560-222', '', '', '', '126203'], ['20240826', 'US Open', 'Hard', 'G', 'W'

In [131]:
def get_player_personal_data(i):
    with open(f"players_data/{i}_{players_without_space[i]}.html") as file:
        soup = BeautifulSoup(file, 'html.parser')
    scripts = soup.find_all('script')
    player_name = scripts[4].text.split("var fullname = '")[1].split("'")[0]
    player_current_rank = scripts[4].text.split('var currentrank = ')[1].split(';')[0]
    player_hand = scripts[4].text.split("var hand = '")[1].split("'")[0]
    player_bday = scripts[4].text.split('var dob = ')[1].split(';')[0]
    player_height = scripts[4].text.split('var ht = ')[1].split(';')[0]
    player_nationality = scripts[4].text.split("var country = '")[1].split("'")[0]
    
    # return {"player_name": player_name, 
    #         "player_current_rank": player_current_rank,
    #         "player_hand": player_hand,
    #         "player_bday": player_bday,
    #         "player_height": player_height,
    #         "player_nationality": player_nationality
    #         }
    return player_name, player_current_rank, player_hand, player_bday, player_height, player_nationality

get_player_personal_data(0)

('Jannik Sinner', '1', 'R', '20010816', '191', 'ITA')

In [132]:
# ['Date', 'Tournament Name', 'Surface', 'Tournament level', 'Win/Loss/Undergoing', 'Rank', 'Seed', '-', 'Round', 'Score', '?', 'Opp Name', 'Opp Rank', 'Opp Seed', '-', 'Opp Hand', 'Opp BDay', 'Opp Height', 'Opp Nationality', '?', ]
# [  0            1                2             3                  4                  5       6     7      8        9      10        11         12          13      14         15      16           17           18             19           20]

In [133]:
def pandas_player_match(player_matches):

    match_data = pd.DataFrame(player_matches, columns=[
        'Date', 
        'Tournament Name', 
        'Surface', 
        'Tournament level', 
        'Status', 
        'Rank', 
        'Seed', 
        'Round', 
        'Score', 
        'Opp Name', 
        'Opp Rank', 
        'Opp Seed', 
        'Opp Hand', 
        'Opp BDay', 
        'Opp Height', 
        'Opp Nationality'
    ])

    match_data['Date'] = pd.to_datetime(match_data['Date'])
    match_data.set_index('Date', inplace=True)
    match_data.sort_index(inplace=True, ascending=False)


    return match_data



In [134]:
class Player:
    def __init__(self, player_name, i):
        self.i = i
        self.player_name = player_name
        self.current_rank = ""
        self.hand = ""
        self.bday = ""
        self.height = ""
        self.nationality = ""
        self.matches = []
        self.matches_pd = None

    def update(self):
        """Update player's personal data using their index."""
        _, self.current_rank, self.hand, self.bday, self.height, self.nationality = get_player_personal_data(self.i)
    
    def update_as_self(self, matchmx_row):
        """Record match data when this player is a participant."""
        relevant_data = [
            matchmx_row[0],  # Date
            matchmx_row[1],  # Tournament Name
            matchmx_row[2],  # Surface
            matchmx_row[3],  # Tournament level
            matchmx_row[4],  # Win/Loss/Undergoing
            matchmx_row[5],  # Rank
            matchmx_row[6],  # Seed
            matchmx_row[8],  # Round
            matchmx_row[9],  # Score
            matchmx_row[11],  # Opponent Name
            matchmx_row[12],  # Opponent Rank
            matchmx_row[13],  # Opponent Seed
            matchmx_row[15],  # Opponent Hand
            matchmx_row[16],  # Opponent BDay
            matchmx_row[17],  # Opponent Height
            matchmx_row[18],  # Opponent Nationality
        ]
        self.matches.append(relevant_data)
    
    def update_as_opp(self, matchmx_row, opp):
        """Record match data when this player is the opponent."""
        status = 'L' if matchmx_row[4] == 'W' else 'W' if matchmx_row[4] == 'L' else 'U'
        relevant_data = [
            matchmx_row[0],  # Date
            matchmx_row[1],  # Tournament Name
            matchmx_row[2],  # Surface
            matchmx_row[3],  # Tournament level
            status,          # Updated status
            matchmx_row[12], # Rank
            matchmx_row[13], # Seed
            matchmx_row[8],  # Round
            matchmx_row[9],  # Score
            opp.player_name,  # Opponent Name
            matchmx_row[5],   # Opponent Rank
            matchmx_row[6],   # Opponent Seed
            opp.hand,         # Opponent Hand
            opp.bday,         # Opponent BDay
            opp.height,       # Opponent Height
            opp.nationality   # Nationality
        ]
        self.matches.append(relevant_data)

    def convert_match_data_to_pd(self):
        """Convert match data to pandas DataFrame."""
        self.matches_pd = pandas_player_match(self.matches)


    def get_surface_win_percentage(self, num):
        """Get the win percentage of the player on a given surface."""
        surface_games = {"Hard": 0, "Clay": 0, "Grass": 0}
        surface_wins = {"Hard": 0, "Clay": 0, "Grass": 0}

        # Count games and wins
        try:
            for i in range(num):
                surface = self.matches[i][2]  # Surface type
                result = self.matches[i][4]    # Match result
                
                if surface in surface_games:
                    surface_games[surface] += 1
                    if result == 'W':
                        surface_wins[surface] += 1
                    elif result == 'U':
                        surface_wins[surface] += 0.5  # Neutral outcome
        except IndexError:
            pass

        # Calculate win percentage
        surface_win_percentage = {}
        for surface in surface_games.keys():
            total_games = surface_games[surface]
            if total_games > 0:
                surface_win_percentage[surface] = (surface_wins[surface] / total_games) * 100
            else:
                surface_win_percentage[surface] = 0.0  # No games played on this surface
    
        return surface_wins, surface_games, surface_win_percentage


    def get_player_meetings(self, player2):
        return self.matches_pd[self.matches_pd['Opp Name'] == player2][['Tournament Name', 
                                                                        'Surface',
                                                                        'Tournament level',
                                                                        'Status',
                                                                        'Round',
                                                                        'Score',
                                                                        'Rank',
                                                                        'Seed',
                                                                        'Opp Rank',
                                                                        'Opp Seed',
                                                                        ]]


In [135]:
get_player_match_data(0)

[['20240926',
  'ATP Beijing',
  'Hard',
  'A',
  'U',
  '1',
  '1',
  '',
  'QF',
  '',
  '3',
  'Jiri Lehecka',
  '37',
  '',
  '',
  'R',
  '20011108',
  '185',
  'CZE',
  '0',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '2',
  '',
  '',
  '',
  '2024-0747-199',
  '',
  '3',
  '3',
  '208103'],
 ['20240926',
  'ATP Beijing',
  'Hard',
  'A',
  'W',
  '1',
  '1',
  '',
  'R16',
  '3-6 6-2 6-3',
  '3',
  'Roman Safiullin',
  '69',
  '',
  'LL',
  'R',
  '19970807',
  '185',
  'RUS',
  '0',
  '141',
  '4',
  '3',
  '77',
  '45',
  '36',
  '15',
  '13',
  '4',
  '6',
  '6',
  '2',
  '93',
  '51',
  '34',
  '19',
  '13',
  '9',
  '13',
  '2',
  '',
  '',
  '',
  '2024-0747-399',
  '',
  '2',
  '2',
  '126128'],
 ['20240926',
  'ATP Beijing',
  'Hard',
  'A',
  'W',
  '1',
  '1',
  '',
  'R32',
  '4-6 6-3 6-1',
  '3',
  'Nicolas Jarry',
  '28',
  '',
  '',
  'R',
  '19951011',
  '201',
  'CHI',
  '0',
  '115',
  '1',


In [136]:
list_all_players = []
for i in range(TPLAYERS):
    if players_with_space[i] not in list_all_players:
        list_all_players.append(players_with_space[i])
    for row in get_player_match_data(i):
        if row[11] not in list_all_players:
            list_all_players.append(row[11])

list_all_players.sort()
print(list_all_players[:10])
len(list_all_players)
    

['Aaro Pollanen', 'Aaron Addison', 'Aaron Cortes Alcaraz', 'Aaron Leeder Chard', 'Abd Hazli Bin Zainuddin', 'Abdelhak Hameurlaine', 'Abdelkader Benabdellah', 'Abdulhamid Mubarak', 'Abdullah Maqdas', 'Abdullah Shannan Alharrasi']


3782

In [137]:
with open("player_names.json", "w") as file:
    json.dump(list_all_players, file)

In [138]:
len(list_all_players)

3782

In [139]:
players = {}
for player in list_all_players:
    num = -1
    if player in players_with_space:
        num = players_with_space.index(player)
    players[player] = Player(player, num)
    if player in players_with_space:
        # print(num)
        players[player].update()
    
    # if player in players_with_space:
    #     players[f"{player}"].update()
    

In [140]:
players['Jannik Sinner'].player_name

'Jannik Sinner'

In [141]:
def get_match_data(i):
    matchmx = get_player_match_data(i)
    # print(matchmx[:5])
    for match_row in matchmx:
        # print(match_row)
        if match_row[11] in players_with_space and players_with_space.index(match_row[11]) <= i:
            # print(match_row[11])
            players[players_with_space[i]].update_as_self(match_row)
        else:
            # print(1, match_row[11])
            players[players_with_space[i]].update_as_self(match_row)
            players[match_row[11]].update_as_opp(match_row, players[players_with_space[i]])


In [142]:
for i in range(TPLAYERS):
    get_match_data(i)

In [143]:
for player_name in list_all_players:
    players[player_name].convert_match_data_to_pd()

In [144]:
def get_player_meetings(player1, player2):
    return players[player1].matches_pd[players[player1].matches_pd['Opp Name'] == player2][['Tournament Name', 
                                                                                            'Surface',
                                                                                            'Tournament level',
                                                                                            'Status',
                                                                                            'Round',
                                                                                            'Score',
                                                                                            'Rank',
                                                                                            'Seed',
                                                                                            'Opp Rank',
                                                                                            'Opp Seed',
                                                                                            ]]


In [145]:
get_player_meetings('Roger Federer', 'Novak Djokovic')

,Tournament Name,Surface,Tournament level,Status,Round,Score,Rank,Seed,Opp Rank,Opp Seed
Date,,,,,,,,,,
2020-01-20,Australian Open,Hard,G,L,SF,7-6(1) 6-4 6-3,3,3,2,2
2019-11-11,Tour Finals,Hard,F,W,RR,6-4 6-3,3,3,2,2
2019-07-01,Wimbledon,Grass,G,L,F,7-6(5) 1-6 7-6(4) 4-6 13-12(3),3,2,1,1
2018-10-29,Paris Masters,Hard,M,L,SF,7-6(6) 5-7 7-6(3),3,3,2,2
2018-08-13,Cincinnati Masters,Hard,M,L,F,6-4 6-4,2,2,10,10
2016-01-18,Australian Open,Hard,G,L,SF,6-1 6-2 3-6 6-3,3,3,1,1
2015-11-15,Tour Finals,Hard,F,L,F,6-3 6-4,3,3,1,1
2015-11-15,Tour Finals,Hard,F,W,RR,7-5 6-2,3,3,1,1
2015-08-31,US Open,Hard,G,L,F,6-4 5-7 6-4 6-4,2,2,1,1


In [146]:
players['Jannik Sinner'].get_surface_win_percentage(100)

({'Hard': 66.5, 'Clay': 11, 'Grass': 10},
 {'Hard': 74, 'Clay': 14, 'Grass': 12},
 {'Hard': 89.86486486486487,
  'Clay': 78.57142857142857,
  'Grass': 83.33333333333334})

In [147]:
get_player_meetings("Novak Djokovic", "Rafael Nadal")

,Tournament Name,Surface,Tournament level,Status,Round,Score,Rank,Seed,Opp Rank,Opp Seed
Date,,,,,,,,,,
2024-07-29,Paris Olympics,Clay,O,W,R32,6-1 6-4,2,1,161,
2022-05-23,Roland Garros,Clay,G,L,QF,6-2 4-6 6-2 7-6(4),1,1,5,5
2021-05-31,Roland Garros,Clay,G,W,SF,3-6 6-3 7-6(4) 6-2,1,1,3,3
2021-05-10,Rome Masters,Clay,M,L,F,7-5 1-6 6-3,1,1,3,2
2020-09-28,Roland Garros,Clay,G,L,F,6-0 6-2 7-5,1,2,2,1
2020-01-06,Atp Cup,Hard,A,W,F,6-2 7-6(4),2,,1,
2019-05-13,Rome Masters,Clay,M,L,F,6-0 4-6 6-1,1,1,2,2
2019-01-14,Australian Open,Hard,G,W,F,6-3 6-2 6-3,1,1,2,2
2018-07-02,Wimbledon,Grass,G,W,SF,6-4 3-6 7-6(9) 3-6 10-8,21,12,1,2


In [148]:
players['Jannik Sinner'].get_player_meetings('Rafael Nadal')

,Tournament Name,Surface,Tournament level,Status,Round,Score,Rank,Seed,Opp Rank,Opp Seed
Date,,,,,,,,,,
2021-05-31,Roland Garros,Clay,G,L,R16,7-5 6-3 6-0,19,18,3,3
2021-05-10,Rome Masters,Clay,M,L,R32,7-5 6-4,18,,3,2
2020-09-28,Roland Garros,Clay,G,L,QF,7-6(4) 6-4 6-1,75,,2,2


In [149]:
import json

# Custom function to convert objects to JSON-serializable dictionaries
def serialize_object(obj):
    if hasattr(obj, '__dict__'):
        # Convert object to a dictionary
        return obj.__dict__
    else:
        return str(obj)  # Or handle objects without __dict__ in a custom way

# Assuming 'players_dict' is your dictionary of objects
with open("players.json", "w") as f:
    json.dump(players, f, default=serialize_object)


import pickle

with open("players.pkl", "wb") as f:
    pickle.dump(players, f)
